## Import

In [1]:
from wmb import cemba
from ALLCools.mcds import MCDS

import pandas as pd
from ALLCools.clustering import log_scale
import glob

In [ ]:
# Parameters
chrom_to_remove = ["chrX", "chrY", "chrM", "chrL"]
cpu = 1
downsample = 1000
mc_type = "CHN"
mem_gb = 1
std_cutoff = 0.05
var_dim = "chrom100k"


In [3]:
_dir = '/home/qzeng_salk_edu/project/230712_m3c-mc-integration'

## Select cells

In [4]:
meta = pd.read_csv(f'{_dir}/m3C_META_230814.csv', index_col = 0)
use_cells = pd.read_csv('m3c_cells.txt', index_col=0, header=None).index
use_cells.name = 'cell'

## Get adata with CEF

In [5]:
mcds_paths = glob.glob(f'{_dir}/female-mcds/pool_amb*.mcds')

mcds = MCDS.open(mcds_paths, var_dim=var_dim, use_obs=use_cells)

mcds.add_mc_frac(
normalize_per_cell=True,  # after calculating mC frac, per cell normalize the matrix
    clip_norm_value=10  # clip outlier values above 10 to 10
)

mcds

<xarray.MCDS>
Dimensions:            (mc_type: 2, cell: 5543, chrom100k: 26350, count_type: 2)
Coordinates:
  * mc_type            (mc_type) <U3 'CGN' 'CHN'
  * cell               (cell) <U43 'AMB_220628_18mo_3F_4E_1_P1-1-I3-A1' ... '...
  * chrom100k          (chrom100k) <U10 'chr1_0' 'chr1_1' ... 'chrX_1710'
    chrom100k_chrom    (chrom100k) <U5 dask.array<chunksize=(13175,), meta=np.ndarray>
    chrom100k_end      (chrom100k) int64 dask.array<chunksize=(26350,), meta=np.ndarray>
    chrom100k_start    (chrom100k) int64 dask.array<chunksize=(26350,), meta=np.ndarray>
  * count_type         (count_type) <U3 'mc' 'cov'
Data variables:
    chrom100k_da       (cell, chrom100k, mc_type, count_type) uint32 dask.array<chunksize=(5, 13175, 1, 1), meta=np.ndarray>
    chrom100k_da_frac  (cell, chrom100k, mc_type) float64 dask.array<chunksize=(5, 13175, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [6]:
mc_cef = pd.read_csv('mC.CEF.csv', header=None, index_col=0).squeeze()
rna_cef = pd.read_csv('m3C.CEF.csv', header=None, index_col=0).squeeze()
cef = mc_cef | rna_cef
cef = cef[cef].index
cef.name = var_dim

In [7]:
mcds.add_feature_selection_column(cef, f'{var_dim}_cef')
adata = mcds.get_adata(mc_type=mc_type,
                       select_hvf=f'{var_dim}_cef',
                       split_large_chunks=False)

In [8]:
std_filter = adata.X.std(axis=0) > std_cutoff
adata._inplace_subset_var(std_filter)

In [9]:
adata

AnnData object with n_obs × n_vars = 5543 × 1328
    var: 'chrom', 'end', 'start', 'cef'

## Preprocessing and save

In [10]:
# log mC fraction and scale features
log_scale(adata, with_mean=True)

StandardScaler()

In [11]:
adata.write_h5ad('m3c_input.h5ad')